In [3]:
!pip install ta


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=d2b4ee78bbca4c23af6d8bf0103e96938e4991d126a184d5b774ff3a180ab3cd
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [18]:
import numpy as np
import pandas as pd
import joblib
import ta
import tensorflow as tf

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_absolute_error

from tensorflow.keras import mixed_precision, optimizers, callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, Dropout, LayerNormalization,
    Conv1D, Add, GlobalAveragePooling1D,
    Concatenate
)

# === 0. TF CONFIGS ===
tf.config.optimizer.set_jit(True)
mixed_precision.set_global_policy('mixed_float16')

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

strategy = tf.distribute.MirroredStrategy()

# === 1. LOAD & CLEAN DATA ===
df = pd.read_csv('/content/MSN.csv', parse_dates=['Date/Time'], on_bad_lines='skip')
df = (
    df.drop(columns=['Ticker', 'Open Interest'], errors='ignore')
      .sort_values('Date/Time')
      .ffill().bfill().dropna().reset_index(drop=True)
)

df['hour'] = df['Date/Time'].dt.hour.astype('int32')
df['roll_mean_30'] = df['Close'].rolling(30).mean().astype('float32')
df['roll_std_30']  = df['Close'].rolling(30).std().astype('float32')
df.dropna(inplace=True)

for col in ['Open', 'High', 'Low', 'Close', 'Volume', 'roll_mean_30', 'roll_std_30']:
    df[col] = df[col].astype('float32')

# === 2. INDICATORS & LAGS ===
lags = [1, 2, 3, 5]
indicators = {
    'RSI': lambda x: ta.momentum.RSIIndicator(x).rsi(),
    'BBw': lambda x: ta.volatility.BollingerBands(x).bollinger_wband(),
    'MACD': lambda x: ta.trend.macd_diff(x),
    'EMA20': lambda x: ta.trend.ema_indicator(x, window=20),
    'ATR': lambda h, l, c: ta.volatility.AverageTrueRange(h, l, c).average_true_range(),
    'ADX': lambda h, l, c: ta.trend.adx(h, l, c),
    'OBV': lambda x, v: ta.volume.on_balance_volume(x, v)
}

for name, fn in indicators.items():
    if name == 'OBV':
        df[name] = fn(df['Close'], df['Volume']).astype('float32')
    elif name in ['ATR', 'ADX']:
        df[name] = fn(df['High'], df['Low'], df['Close']).astype('float32')
    else:
        df[name] = fn(df['Close']).astype('float32')

for lag in lags:
    df[f'C_lag{lag}'] = df['Close'].shift(lag)
    df[f'V_lag{lag}'] = df['Volume'].shift(lag)

df.dropna(inplace=True)

# === 3. MULTI-STEP TARGET ===
H = 5
for h in range(1, H + 1):
    df[f'Fut{h}'] = df['Close'].shift(-h)

df.dropna(inplace=True)
Y_all = np.stack([
    df[f'Fut{h}'].to_numpy() - df['Close'].to_numpy() for h in range(1, H + 1)
], axis=1).astype('float32')

# === 4. FEATURE LISTS ===
static_feats = [
    'Open', 'High', 'Low', 'Close', 'Volume',
    'roll_mean_30', 'roll_std_30', 'hour',
    *indicators.keys(),
    *[f'C_lag{lag}' for lag in lags],
    *[f'V_lag{lag}' for lag in lags]
]

seq_feats = ['Close', 'Volume', 'RSI', 'BBw', 'MACD', 'EMA20', 'ATR', 'ADX', 'OBV']
window = 30

# === 5. BUILD SEQUENCES ===
def build_seq(df, window):
    arr_s = df[static_feats].to_numpy(dtype='float32')
    arr_q = df[seq_feats].to_numpy(dtype='float32')
    S, Q = [], []
    for i in range(window, len(df) - H + 1):
        S.append(arr_s[i])
        Q.append(arr_q[i - window:i])
    return np.array(S), np.array(Q)

Xs, Xq = build_seq(df, window)
N = Xs.shape[0]
Y = Y_all[window:window + N]

# === 6. SPLIT & SCALE ===
iT = int(N * 0.8)
iV = int(iT * 0.8)

Xs_tr, Xs_val, Xs_te = Xs[:iV], Xs[iV:iT], Xs[iT:]
Xq_tr, Xq_val, Xq_te = Xq[:iV], Xq[iV:iT], Xq[iT:]
Y_tr, Y_val, Y_te = Y[:iV], Y[iV:iT], Y[iT:]

scaler_s = StandardScaler().fit(Xs_tr)
scaler_q = StandardScaler().fit(Xq_tr.reshape(-1, len(seq_feats)))
scaler_tar = RobustScaler().fit(Y_tr)

def scale(Xs, Xq):
    xs = scaler_s.transform(Xs)
    xq = scaler_q.transform(Xq.reshape(-1, len(seq_feats))).reshape(Xq.shape)
    return xs, xq

Xs_tr_s, Xq_tr_s = scale(Xs_tr, Xq_tr)
Xs_val_s, Xq_val_s = scale(Xs_val, Xq_val)
Xs_te_s, Xq_te_s = scale(Xs_te, Xq_te)

# === 7. TF DATASETS ===
bs = 256
def mkds(Xs, Xq, Y, shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices(((Xs, Xq), Y))
    if shuffle:
        ds = ds.shuffle(buffer_size=10000)
    return ds.batch(bs).prefetch(tf.data.AUTOTUNE)

ds_tr = mkds(Xs_tr_s, Xq_tr_s, Y_tr, True)
ds_val = mkds(Xs_val_s, Xq_val_s, Y_val)

# === 8. TCN BLOCK ===
def TCN(x, filters, kernel, dil):
    y = Conv1D(filters, kernel, dilation_rate=dil, padding='causal', activation='relu')(x)
    y = Conv1D(filters, kernel, dilation_rate=dil, padding='causal', activation='relu')(y)
    return Add()([x, y])

# === 9. MODEL ===
with strategy.scope():
    def build():
        inp_s = Input((len(static_feats),), name='static_input')
        xs = Dense(64, activation='relu')(inp_s)
        xs = Dropout(0.3)(xs)

        inp_q = Input((window, len(seq_feats)), name='seq_input')
        x = Dense(128)(inp_q)
        for d in [1, 2, 4, 8, 16]:
            x = TCN(x, 128, 3, d)
        x = LayerNormalization()(x)
        enc = GlobalAveragePooling1D()(x)

        merged = Concatenate()([xs, enc])
        h = Dense(128, activation='relu')(merged)
        h = Dropout(0.2)(h)
        out = Dense(H, name='reg')(h)

        model = Model([inp_s, inp_q], out)
        model.compile(
            optimizer=optimizers.AdamW(learning_rate=5e-5, weight_decay=1e-4),
            loss='mae',
            metrics=[tf.keras.metrics.MeanAbsoluteError()]
        )
        return model

    model = build()
    es = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
    hist = model.fit(ds_tr, validation_data=ds_val, epochs=50, callbacks=[es, rl], verbose=2)

# === 10. EVALUATE ===
pred = model.predict([Xs_te_s, Xq_te_s], batch_size=bs)
pred = scaler_tar.inverse_transform(pred)

for h in range(H):
    print(f"Step {h+1} MAE = {mean_absolute_error(Y_te[:, h], pred[:, h]):.4f}")

# === 11. SAVE MODEL ===
model.save('best_tcn_enhanced.keras')
joblib.dump(scaler_s, 'stat_s.pkl')
joblib.dump(scaler_q, 'seq_s.pkl')
joblib.dump(scaler_tar, 'tar_s.pkl')


Epoch 1/50
339/339 - 15s - 44ms/step - loss: 0.2144 - mean_absolute_error: 0.2144 - val_loss: 0.1136 - val_mean_absolute_error: 0.1136 - learning_rate: 5.0000e-05
Epoch 2/50
339/339 - 8s - 25ms/step - loss: 0.1523 - mean_absolute_error: 0.1523 - val_loss: 0.1103 - val_mean_absolute_error: 0.1103 - learning_rate: 5.0000e-05
Epoch 3/50
339/339 - 3s - 10ms/step - loss: 0.1497 - mean_absolute_error: 0.1497 - val_loss: 0.1089 - val_mean_absolute_error: 0.1089 - learning_rate: 5.0000e-05
Epoch 4/50
339/339 - 4s - 11ms/step - loss: 0.1491 - mean_absolute_error: 0.1491 - val_loss: 0.1079 - val_mean_absolute_error: 0.1078 - learning_rate: 5.0000e-05
Epoch 5/50
339/339 - 5s - 14ms/step - loss: 0.1487 - mean_absolute_error: 0.1486 - val_loss: 0.1071 - val_mean_absolute_error: 0.1071 - learning_rate: 5.0000e-05
Epoch 6/50
339/339 - 3s - 10ms/step - loss: 0.1486 - mean_absolute_error: 0.1486 - val_loss: 0.1065 - val_mean_absolute_error: 0.1065 - learning_rate: 5.0000e-05
Epoch 7/50
339/339 - 4s - 1

['tar_s.pkl']